In [ ]:
import combinatorial_gwas.high_level as cgwas
import tensorflow as tf
import kerastuner as kt
import sys
import os
import shutil
import string
import sklearn

/lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.ven

## Useful Constants

* A = number of different genotypic alleles
* P = number of different phenotypes
* D = dropout rate
* l2 = L2 regularization rate

## Network Architecture
Outputs:

1. Input: N x A x 3
1. Dense Layer w/ ReLU: N x A // 3
1. Dropout(D): N x A // 3
1. Dense Layer w/ ReLU: N x P * 3
1. Dense Layer w/ Sigmoid: N x P

In [ ]:
def make_DNN_model(d1, dropout, l2, A, P):
    regularizer = tf.keras.regularizers.L2(l2)
    return tf.keras.Sequential(layers=[
        tf.keras.layers.InputLayer(input_shape=(A, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(d1, activation=tf.nn.relu, kernel_regularizer=regularizer),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(P, activation=tf.nn.sigmoid, kernel_regularizer=regularizer)
    ], name='nn_model')

## Model Training Parameters
#### Loss Function
Binary Cross-Entropy + L2 Regularization
#### Optimizer
Adam
#### Metric
Binary Accuracy

In [ ]:
def compile_model(model, optimizer):
    model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.metrics.AUC()])
    model.summary()

## Hyperparameter Tuning

In [ ]:
batch_size = 100
epochs = 20
use_multiprocessing=False
validation_split = 0.2

def get_tunable_DNN_model(hp, d1_min, d1_max, d_step, A, P):
    d1_units = hp.Int('d1_units', min_value=d1_min, max_value=d1_max, step=d_step)
    dropout = hp.Choice('dropout', values=[0.2, 0.5, 0.8])
    l2 = hp.Choice('l2', values=[0.01, 0.005, 0.001])
    lr = hp.Choice('lr', values=[0.01, 0.005, 0.001])
    model = make_DNN_model(d1_units, dropout, l2, A, P)
    compile_model(model, tf.keras.optimizers.Adam(lr))
    return model

def get_tuner(model_builder):
    try:
      shutil.rmtree('hp_tuning')
    except FileNotFoundError:
      pass
    return kt.Hyperband(model_builder, objective=kt.Objective('val_auc', direction='max'), max_epochs=epochs, executions_per_trial=3, directory='hp_tuning', project_name='initial_model')

## Search Space

In [ ]:
d1_min = max(P // 4, 1)
d1_max = P * 64
d_step = 16

## Get Training Data

In [ ]:
#def get_data(N, A, P):
 #   genotypes = tf.stack([tf.constant([[1, 0, 0]] * A, dtype=tf.float32)] * N)
  #  phenotypes = tf.stack([tf.constant([1] * P, dtype=tf.float32)] * N)
   # return genotypes, phenotypes

## Hyperparameter Tuning Call

In [ ]:
datasource = cgwas.chromosome_datasource(21)

Sample IDs are read from /lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr21_v3_s487275.sample.


Mapping genotypes: 100%|██████████| 1261158/1261158 [01:09<00:00, 18040.30it/s]


In [ ]:
search_data = datasource.get_data(snp_filters=[cgwas.snp_filter('R07', cgwas.snp_filter.SORT_PVALUE)], max_samples=1000)
search_train_x, search_validation_x, search_train_y, search_validation_y = sklearn.model_selection.train_test_split(*search_data, test_size=0.2, random_state=0, stratify=search_data[1])

reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1


In [ ]:
search_train_x, search_validation_x, search_train_y, search_validation_y = sklearn.model_selection.train_test_split(*search_data, test_size=0.2, random_state=0, stratify=search_data[1])
np.set_printoptions(threshold=1000)
count = 0
for i in range(search_train_x.shape[0]):
    for j in range(search_train_x.shape[1]):
        if search_train_x[i][j][0] != 1:
            count += 1
search_train_y = np.array(search_train_y)
count2 = 0
for i in range(search_train_y.shape[0]):
    if search_train_y[i]:
        count2 += 1
print(count, count2)
print(search_train_x.shape[1])

353 59
1


In [ ]:
tuner = get_tuner(lambda hp: get_tunable_DNN_model(hp, d1_min, d1_max, d_step, search_train_x.shape[1], search_train_y.shape[1]))
tuner.search_space_summary()
tuner.search(search_train_x, search_train_y, epochs=epochs, validation_data=(search_validation_x, search_validation_y))
tuner.results_summary()

Trial 30 Complete [00h 00m 33s]
val_auc: 0.48270271221796673

Best val_auc So Far: 0.5331531961758932
Total elapsed time: 00h 14m 35s
Results summary
Results in hp_tuning/initial_model
Showing 10 best trials
Objective(name='val_auc', direction='max')
Trial summary
Hyperparameters:
d1_units: 17
dropout: 0.8
l2: 0.001
lr: 0.001
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 6c0de381e3c2b88bbcb0dda6564e6dda
Score: 0.5331531961758932
Trial summary
Hyperparameters:
d1_units: 33
dropout: 0.8
l2: 0.001
lr: 0.01
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.5302702784538269
Trial summary
Hyperparameters:
d1_units: 33
dropout: 0.2
l2: 0.001
lr: 0.01
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.5237837831179301
Trial summary
Hyperparameters:
d1_units: 33
dropout: 0.5
l2: 0.001
lr: 0.01
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.5237837831179301
Tria

## Optimal Model

In [ ]:
models = tuner.get_best_models()

Model: "nn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3000)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                51017     
_________________________________________________________________
dropout (Dropout)            (None, 17)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 18        
Total params: 51,035
Trainable params: 51,035
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [ ]:
x, y = input_processing.get_data(A, 100000)
train_x, validation_x, train_y, validation_y = sklearn.model_selection.train_test_split(x, y, test_size=validation_split, stratify=y)

In [ ]:
histories = [model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epochs, validation_data=(validation_x, validation_y), use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1) for model in models]

Epoch 1/20
800/800 [==============================] - 5s 6ms/step - loss: 0.3695 - accuracy: 0.9627 - auc: 0.5210 - val_loss: 0.1787 - val_accuracy: 0.9622 - val_auc: 0.5146
Epoch 2/20
800/800 [==============================] - 4s 5ms/step - loss: 0.2390 - accuracy: 0.9611 - auc: 0.4999 - val_loss: 0.1668 - val_accuracy: 0.9622 - val_auc: 0.5318
Epoch 3/20
800/800 [==============================] - 4s 5ms/step - loss: 0.1965 - accuracy: 0.9630 - auc: 0.5015 - val_loss: 0.1670 - val_accuracy: 0.9622 - val_auc: 0.5043
Epoch 4/20
800/800 [==============================] - 4s 5ms/step - loss: 0.1795 - accuracy: 0.9626 - auc: 0.5049 - val_loss: 0.1658 - val_accuracy: 0.9622 - val_auc: 0.5071
Epoch 5/20
800/800 [==============================] - 4s 5ms/step - loss: 0.1726 - accuracy: 0.9617 - auc: 0.5025 - val_loss: 0.1653 - val_accuracy: 0.9622 - val_auc: 0.5091
Epoch 6/20
800/800 [==============================] - 4s 5ms/step - loss: 0.1662 - accuracy: 0.9620 - auc: 0.5061 - val_loss: 0.16

## Test Model

In [ ]:
test_x, test_y = input_processing.get_data(A, 1000)

NameError: ignored

In [ ]:
model = models[0]
history = model.evaluate(x=test_x, y=test_y, use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1)

32/32 [==============================] - 0s 3ms/step - loss: 0.1162 - accuracy: 0.9760 - auc: 0.5000


In [ ]:
model.save('/content/gdrive/My Drive/6.874 project/Colab/models/dnn')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/6.874 project/Colab/models/dnn/assets
